# Algoritmo para identificação de pessoas e captura da cor de suas roupas usando apenas métodos de visão computacional

In [1]:
import cv2 as cv
import numpy as np
import time

# Abrir o vídeo
cam = cv.VideoCapture("./video1.mp4")

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
false_id = 500000
TIMEOUT = 25  # Número de frames até considerarmos que a pessoa saiu
DETECTION_DELAY = 100  # Número de frames para considerar uma nova pessoa
paused = False
prev_time = time.time()
orb = cv.ORB_create()

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()


frame_reference_gray = cv.cvtColor(frame_reference, cv.COLOR_BGR2GRAY)
frame_reference_gray = cv.GaussianBlur(frame_reference_gray, (31, 31), 0)

COLOR_NAMES = {
    'preto': (0, 0, 0),
    'branco': (255, 255, 255),
    'cinza escuro': (64, 64, 64),
    'cinza': (128, 128, 128),
    'cinza claro': (192, 192, 192),
    'vermelho': (255, 0, 0),
    'verde': (0, 255, 0),
    'azul': (0, 0, 255),
    'amarelo': (255, 255, 0),
    'ciano': (0, 255, 255),
    'magenta': (255, 0, 255),
    'laranja': (255, 165, 0),
    'rosa': (255, 192, 203),
    'roxo': (128, 0, 128),
    'marrom': (165, 42, 42),
    'azul claro': (173, 216, 230),
    'verde claro': (144, 238, 144),
    'vermelho claro': (255, 102, 102),
    'amarelo claro': (255, 255, 153),
    'laranja escuro': (255, 140, 0),
    'roxo claro': (216, 191, 216),
    'azul escuro': (0, 0, 139),
    'verde escuro': (0, 100, 0),
    'vermelho escuro': (139, 0, 0),
    'dourado': (255, 215, 0),
    'prata': (192, 192, 192),
    'champanhe': (247, 231, 206),
    'violeta': (238, 130, 238),
    'salmão': (250, 128, 114),
    'menta': (189, 252, 201),
    'lavanda': (230, 230, 250),
    'azul marinho': (0, 0, 128),
    'verde oliva': (107, 142, 35),
    'bege': (245, 245, 220),
    'turquesa': (64, 224, 208),
    'marfim': (255, 255, 240)
}


# Função para calcular o histograma de cores de um ROI
def get_color_histogram(roi):
    hist = cv.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv.normalize(hist, hist).flatten()


# Função para calcular a distância euclidiana entre cores
def euclidean_distance(color1, color2):
    return np.sqrt(np.sum((np.array(color1) - np.array(color2)) ** 2))

# Função para encontrar o nome da cor mais próxima
def get_color_name(mean_color):
    closest_color = None
    min_distance = float('inf')
    for color_name, color_value in COLOR_NAMES.items():
        dist = euclidean_distance(mean_color[:3], color_value)  # Ignorar o canal alpha (transparência)
        if dist < min_distance:
            min_distance = dist
            closest_color = color_name
    return closest_color


def get_median_color(roi_contour, mask):
    # Extraímos as cores do ROI usando a máscara
    roi_colors = roi_contour[mask == 255]  # Pega apenas os pixels dentro da máscara
    if roi_colors.size == 0:
        return (0, 0, 0)  # Retorna preto se não houver pixels válidos

    # Calcula a mediana para cada canal (B, G, R)
    median_color = np.median(roi_colors, axis=0)
    return median_color


while True:
    if not paused:
        ret, frame = cam.read()

        if not ret:
            break
        
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame_gray = cv.GaussianBlur(frame_gray, (31, 31), 0)

        # Subtrair o frame de referência do frame atual
        frame_delta = cv.absdiff(frame_reference_gray, frame_gray)
        _, thresh = cv.threshold(frame_delta, 25, 255, cv.THRESH_BINARY)
        thresh = cv.dilate(thresh, None, iterations=2)
        
        colored_movement = cv.bitwise_and(frame, frame, mask=thresh)

        # Encontrar contornos
        contours, _ = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # Filtrar contornos pequenos e criar uma lista de áreas
        valid_contours = []
        areas = [cv.contourArea(contour) for contour in contours if cv.contourArea(contour) >= 1000]

        if areas:
            max_area = max(areas)

            for contour in contours:
                area = cv.contourArea(contour)
                if area >= 1000 and area > max_area * 0.3:
                    valid_contours.append(contour)

        # Loop sobre os contornos filtrados
        for contour in valid_contours:
            x, y, w, h = cv.boundingRect(contour)
            roi = frame[y:y+h, x:x+w]
            person_hist = get_color_histogram(roi)

            new_person = True
            for person in people_detected:
                dist = cv.compareHist(person['hist'], person_hist, cv.HISTCMP_CORREL)
                if dist > 0.8:
                    person['last_seen'] = (x, y)
                    person['missed_frames'] = 0
                    person['detection_delay'] += 1  # Incrementa o contador de frames para essa pessoa
                    new_person = False
                    
                    # Se a pessoa já estiver detectada há mais de 3 segundos, capturar keypoints e descritores
                    if time.time() - person['time_entered'] > 3:
                        if person['false_id'] != 0: 
                            person_id += 1
                            person["real_id"] = person_id
                        person['false_id'] = 0

                        hull = cv.convexHull(contour)
                        cv.drawContours(frame, [hull], -1, (0, 255, 0), 2)

                        kp = orb.detect(colored_movement, None) 
                        kp, des = orb.compute(colored_movement, kp)
                        person['keypoints'] = kp[:person['num_descriptors']]
                        person['descriptors'] = des[:person['num_descriptors']]
                        person['num_descriptors'] += 10  # Aumenta o número de descritores capturados

                        # Desenhar keypoints na imagem
                        frame = cv.drawKeypoints(frame, kp, 0, (0, 0, 255), flags=cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
                        cv.putText(frame, f"Pessoa {person['real_id']} detectada", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Adicionar o código para calcular a cor predominante dentro do contorno
                    roi_contour = frame[y:y+h, x:x+w]
                    mask = np.zeros_like(roi_contour)
                    
                    cv.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv.FILLED, offset=(-x, -y))
                    median_color = get_median_color(roi_contour, mask[:, :, 0])
                    median_color = [float(f"{median_color[0]:.1f}"), float(f"{median_color[1]:.1f}"), float(f"{median_color[2]:.1f}")]
                    color_name = get_color_name(median_color)
                    # cv.putText(frame, f"Cor: {color_name}", (x, y + 30), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                    
                    break

            if new_person:
                false_id += 1
                people_detected.append({
                    'false_id': false_id,
                    'real_id': false_id,
                    'hist': person_hist,
                    'last_seen': (x, y),
                    'missed_frames': 0,
                    'detection_delay': 1,  # Inicia o contador para nova detecção
                    'time_entered': time.time(),
                    'num_descriptors': 10,  # Inicia com 10 descritores
                    'keypoints': None,
                    'descriptors': None
                })

        # Atualiza o contador de frames perdidos para todas as pessoas
        for person in people_detected:
            if person['missed_frames'] > TIMEOUT:
                people_detected.remove(person)
            else:
                person['missed_frames'] += 1

        current_time = time.time()
        fps = 1 / (current_time - prev_time)
        prev_time = current_time
        
        cv.putText(frame, f'FPS: {int(fps)}', (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Mostrar IDs de todas as pessoas detectadas recentemente
        cv.imshow('Colored', colored_movement)
        cv.imshow('frame delta', frame_delta)
        cv.imshow('Deteccao de Mudancas', frame)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

# Liberar o vídeo e fechar as janelas
cam.release()
cv.destroyAllWindows()


# Algoritmo para identificação de pessoas e captura da cor de suas roupas usando YOLO

In [ ]:
!pip install --quiet ultralytics
!pip install --quiet scikit-learn

from ultralytics import YOLO
import cv2

model = YOLO('yolov8n.pt')


In [ ]:
from ultralytics import YOLO
import cv2
from collections import defaultdict
import numpy as np
import time
from sklearn.cluster import KMeans

cap = cv2.VideoCapture("./video2.mp4")

# Usa modelo da Yolo
model = YOLO("yolov8n.pt")  # Modelo leve para melhor desempenho

track_history = defaultdict(lambda: [])
person_id_counter = 0  # Contador de IDs de pessoas
person_ids = {}  # Dicionário para mapear track_id para um person_id exclusivo

seguir = True
deixar_rastro = True

frame_count = 0
frame_skip = 2  # Ajuste o valor para pular mais frames e melhorar o desempenho

# Iniciar o contador de tempo para o cálculo do FPS
prev_time = time.time()

# Reduz a resolução para aumentar o FPS (opcional)
resize_width = 640
resize_height = 360

# Função para obter a cor predominante
def get_dominant_color(image, box):
    x, y, w, h = map(int, box)
    person_roi = image[y - h // 2: y + h // 2, x - w // 2: x + w // 2]

    if person_roi.size == 0:
        return (0, 0, 0)  # Evitar erro se a ROI estiver vazia

    # Redimensiona a imagem para facilitar o cálculo da cor dominante
    person_roi = cv2.resize(person_roi, (50, 50), interpolation=cv2.INTER_AREA)
    
    # Converte a imagem para um array 2D de pixels
    pixels = person_roi.reshape((-1, 3))
    
    # Usa o KMeans para encontrar a cor mais comum
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(pixels)
    
    # A cor predominante será o centro do cluster
    dominant_color = kmeans.cluster_centers_[0].astype(int)
    
    return tuple(map(int, dominant_color))  # Certifique-se de retornar inteiros

while True:
    success, img = cap.read()

    if not success:
        break  # Sai do loop se não conseguir ler mais frames

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue  # Pule este frame para melhorar o desempenho

    # Redimensiona o frame para resolução menor
    img = cv2.resize(img, (resize_width, resize_height))

    # YOLOv8 Processamento
    if seguir:
        results = model.track(img, persist=True)
    else:
        results = model(img)

    # Cria uma cópia da imagem original para desenhar apenas pessoas
    img_people_only = img.copy()

    # Processar a lista de resultados
    for result in results:
        if seguir and deixar_rastro:
            try:
                person_class_index = 0  # Índice da classe "person" na YOLOv8 é geralmente 0
                boxes = result.boxes.xywh.cpu()
                track_ids = result.boxes.id.int().cpu().tolist()
                class_ids = result.boxes.cls.int().cpu().tolist()

                # Desenha as caixas, IDs e rastros apenas para pessoas
                for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                    if class_id == person_class_index:
                        x, y, w, h = map(int, box)  # Certifique-se de que as coordenadas são inteiras

                        # Atribui um novo person_id se ainda não estiver no dicionário
                        if track_id not in person_ids:
                            person_ids[track_id] = person_id_counter
                            person_id_counter += 1

                        person_id = person_ids[track_id]

                        # Obtém a cor predominante da pessoa
                        dominant_color = get_dominant_color(img, box)

                        # Desenha o retângulo ao redor das pessoas
                        cv2.rectangle(img_people_only, (x - w // 2, y - h // 2),
                                      (x + w // 2, y + h // 2), (0, 255, 0), 2)

                        # Adiciona o ID e a cor predominante na imagem perto do retângulo
                        cv2.putText(img_people_only, f'ID: {person_id}', (x - w // 2, y - h // 2 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                        cv2.putText(img_people_only, f'Cor BGR {dominant_color}', (x - w // 2, y - h // 2 - 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, ((dominant_color[0]), int(dominant_color[1]), int(dominant_color[2])), 2)

                        # Rastreia o histórico de posição para cada pessoa
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  # Ponto central x, y
                        if len(track) > 30:  # Mantém no máximo 30 frames no histórico
                            track.pop(0)

                        # Desenha as linhas de rastreamento
                        points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
                        cv2.polylines(img_people_only, [points], isClosed=False, color=(230, 0, 0), thickness=5)

            except Exception as e:
                print(f"Erro: {e}")
                pass

    # Calcula o FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Desenha o FPS no canto da imagem
    cv2.putText(img_people_only, f'FPS: {int(fps)}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Mostra a imagem filtrada com apenas pessoas
    cv2.imshow("Frame", img_people_only)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("desligando")
